https://stackoverflow.com/questions/77834567/how-to-use-shared-memory-in-pycuda-logicerror-cumoduleloaddataex-failed-an-il

In [2]:
import pycuda.driver as drv
import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

n = 20

input = np.random.randint(10, size=(n))
output = np.zeros_like(input)

input = input.astype(np.int32)
output = output.astype(np.int32)

mod = SourceModule(
'''
  __global__ void flipVectorSM(int* in, int* out, int n) {
    extern __shared__ int sData[];
    int inOffSet = blockDim.x * blockIdx.x;
    int index = inOffSet + threadIdx.x;
    if (index < n) {
        sData[blockDim.x - 1 - threadIdx.x] = in[index];
        __syncthreads();
    }
    int outOffSet = blockDim.x * (gridDim.x - 1 - blockIdx.x);
    int outIndex = outOffSet + threadIdx.x;
    out[outIndex] = sData[threadIdx.x];
  }
'''
)

flip = mod.get_function('flipVectorSM')
flip(drv.In(input), drv.InOut(output), np.int32(n), block=(20, 1, 1), grid=(1, 1), shared=20)

print("Input vector:")
print(input)
print("\nOutput vector:")
print(output)

Input vector:
[0 4 1 4 5 7 4 9 8 8 8 1 8 7 5 4 5 7 8 3]

Output vector:
[3 8 7 5 4 5 7 8 1 8 8 8 9 4 7 5 4 1 4 0]


In [10]:
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np

# CUDA kernel for static memory version
static_reverse_kernel = """
__global__ void staticReverse(int *d, int n)
{
  __shared__ int s[64];
  int t = threadIdx.x;
  int tr = n-t-1;
  s[t] = d[t];
  __syncthreads();
  d[t] = s[tr];
}
"""

# CUDA kernel for dynamic memory version
dynamic_reverse_kernel = """
__global__ void dynamicReverse(int *d, int n)
{
  extern __shared__ int s[];
  int t = threadIdx.x;
  int tr = n-t-1;
  s[t] = d[t];
  __syncthreads();
  d[t] = s[tr];
}
"""

# Compile the CUDA kernels
mod_static = SourceModule(static_reverse_kernel)
mod_dynamic = SourceModule(dynamic_reverse_kernel)

# Get the kernel functions
static_reverse_func = mod_static.get_function("staticReverse")
dynamic_reverse_func = mod_dynamic.get_function("dynamicReverse")

# Host data
n = 64
a = np.arange(n).astype(np.int32)
r = np.empty_like(a)
d = cuda.mem_alloc(a.nbytes)

# Copy data to device
cuda.memcpy_htod(d, a)

# Run static version kernel
static_reverse_func(d, np.int32(n), block=(n, 1, 1))
cuda.memcpy_dtoh(r, d)
print("Static Reverse Result:", r)

# Run dynamic version kernel
dynamic_reverse_func(d, np.int32(n), block=(n, 1, 1), shared=4*n)  # 4*n bytes of shared memory for int data
cuda.memcpy_dtoh(r, d)
print("Dynamic Reverse Result:", r)

Static Reverse Result: [63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40
 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16
 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]
Dynamic Reverse Result: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]


In [11]:
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63], dtype=int32)

In [12]:
dynamic_reverse_kernel = """
__global__ void dynamicReverse(int *d, int n)
{
  extern __shared__ int s[];
  int t = threadIdx.x;
  int tr = n-t-1;
  s[t] = d[t];
  __syncthreads();
  d[t] = s[tr];
}
"""

# Compile the CUDA kernels
mod_dynamic = SourceModule(dynamic_reverse_kernel)

# Get the kernel functions
dynamic_reverse_func = mod_dynamic.get_function("dynamicReverse")

# Host data
n = 64
a = np.arange(n).astype(np.int32)
r = np.empty_like(a)
d = cuda.mem_alloc(a.nbytes)

# Copy data to device
cuda.memcpy_htod(d, a)

# Run dynamic version kernel
dynamic_reverse_func(d, np.int32(n), block=(n, 1, 1), shared=n * 4)  # Allocate shared memory per thread
cuda.memcpy_dtoh(r, d)
print("Dynamic Reverse Result:", r)

Dynamic Reverse Result: [63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45 44 43 42 41 40
 39 38 37 36 35 34 33 32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17 16
 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]


In [1]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

# Kernel function for vector addition
kernel_code = """
__global__ void vector_add(float *a, float *b, float *result, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n)
        result[i] = a[i] + b[i];
}
"""

# Compile the kernel code
mod = SourceModule(kernel_code)

# Get the kernel function
vector_add = mod.get_function("vector_add")

# Define the size of the vectors
n = 1000

# Create random input data
a = np.random.randn(n).astype(np.float32)
b = np.random.randn(n).astype(np.float32)

# Allocate memory on the GPU
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
result_gpu = cuda.mem_alloc(a.nbytes)

# Copy data to GPU
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# Define block and grid size
block_size = 256
grid_size = (n + block_size - 1) // block_size

# Call the kernel function
vector_add(a_gpu, b_gpu, result_gpu, np.int32(n), block=(block_size, 1, 1), grid=(grid_size, 1))

# Allocate memory for the result on the host
result = np.empty_like(a)

# Copy the result back to the host
cuda.memcpy_dtoh(result, result_gpu)

# Verify the result
expected_result = a + b
assert np.allclose(result, expected_result)

print("Vector addition successfully performed on GPU.")


Vector addition successfully performed on GPU.


In [4]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

# CUDA kernel for 2D convolution
convolution_kernel = """
__global__ void convolution(float *input, float *output, float *kernel, int input_width, int input_height, int kernel_width, int kernel_height)
{
    int tx = threadIdx.x + blockDim.x * blockIdx.x;
    int ty = threadIdx.y + blockDim.y * blockIdx.y;

    if (tx < input_width && ty < input_height)
    {
        int output_index = ty * input_width + tx;

        output[output_index] = 0.0;
        for (int i = 0; i < kernel_height; ++i)
        {
            for (int j = 0; j < kernel_width; ++j)
            {
                int input_index = (ty + i - kernel_height / 2) * input_width + (tx + j - kernel_width / 2);
                int kernel_index = i * kernel_width + j;
                if (input_index >= 0 && input_index < input_width * input_height)
                {
                    output[output_index] += input[input_index] * kernel[kernel_index];
                }
            }
        }
    }
}
"""

def conv2d(input_tensor, kernel):
    input_height, input_width = input_tensor.shape
    kernel_height, kernel_width = kernel.shape

    # Allocate device memory
    input_gpu = cuda.mem_alloc(input_tensor.nbytes)
    output_gpu = cuda.mem_alloc(input_tensor.nbytes)
    kernel_gpu = cuda.mem_alloc(kernel.nbytes)

    # Copy data to device
    cuda.memcpy_htod(input_gpu, input_tensor)
    cuda.memcpy_htod(kernel_gpu, kernel)

    # Define block and grid dimensions
    block_size = (16, 16, 1)
    grid_size = ((input_width + block_size[0] - 1) // block_size[0],
                 (input_height + block_size[1] - 1) // block_size[1])

    # Compile CUDA kernel
    mod = SourceModule(convolution_kernel)
    convolution_func = mod.get_function("convolution")

    # Execute the kernel
    convolution_func(input_gpu, output_gpu, kernel_gpu,
                      np.int32(input_width), np.int32(input_height),
                      np.int32(kernel_width), np.int32(kernel_height),
                      block=block_size, grid=grid_size)

    # Copy the result back to host
    output_tensor = np.empty_like(input_tensor)
    cuda.memcpy_dtoh(output_tensor, output_gpu)

    return output_tensor

# Example usage:
input_tensor = np.random.rand(32, 32).astype(np.float32)
kernel = np.random.rand(3, 3).astype(np.float32)

output_tensor = conv2d(input_tensor, kernel)
print("Output Tensor shape:", output_tensor.shape)


Output Tensor shape: (32, 32)


In [5]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

# CUDA kernel for Non-Maximum Suppression
nms_kernel = """
__global__ void nms(float *boxes, int *selected_indices, int num_boxes, float threshold)
{
    int idx = threadIdx.x + blockDim.x * blockIdx.x;

    if (idx < num_boxes)
    {
        float x1 = boxes[idx * 4];
        float y1 = boxes[idx * 4 + 1];
        float x2 = boxes[idx * 4 + 2];
        float y2 = boxes[idx * 4 + 3];
        float area = (x2 - x1 + 1) * (y2 - y1 + 1);

        for (int i = 0; i < num_boxes; ++i)
        {
            if (i != idx)
            {
                float xx1 = fmaxf(x1, boxes[i * 4]);
                float yy1 = fmaxf(y1, boxes[i * 4 + 1]);
                float xx2 = fminf(x2, boxes[i * 4 + 2]);
                float yy2 = fminf(y2, boxes[i * 4 + 3]);

                float w = fmaxf(0.0f, xx2 - xx1 + 1);
                float h = fmaxf(0.0f, yy2 - yy1 + 1);
                float overlap = (w * h) / area;

                if (overlap > threshold)
                {
                    return; // Suppress this box
                }
            }
        }

        selected_indices[idx] = 1; // Keep this box
    }
}
"""

def nms_cuda(boxes, threshold):
    num_boxes = boxes.shape[0]
    selected_indices = np.zeros(num_boxes, dtype=np.int32)

    # Allocate device memory
    boxes_gpu = cuda.mem_alloc(boxes.nbytes)
    selected_indices_gpu = cuda.mem_alloc(selected_indices.nbytes)

    # Copy data to device
    cuda.memcpy_htod(boxes_gpu, boxes)
    cuda.memcpy_htod(selected_indices_gpu, selected_indices)

    # Define block and grid dimensions
    block_size = 256
    grid_size = (num_boxes + block_size - 1) // block_size

    # Compile CUDA kernel
    mod = SourceModule(nms_kernel)
    nms_func = mod.get_function("nms")

    # Execute the kernel
    nms_func(boxes_gpu, selected_indices_gpu, np.int32(num_boxes), np.float32(threshold),
             block=(block_size, 1, 1), grid=(grid_size, 1))

    # Copy the result back to host
    cuda.memcpy_dtoh(selected_indices, selected_indices_gpu)

    return selected_indices

# Example usage:
boxes = np.array([[10, 10, 50, 50], [20, 20, 60, 60], [30, 30, 70, 70]], dtype=np.float32)
threshold = 0.5

selected_indices = nms_cuda(boxes, threshold)
print("Selected Indices after NMS:", selected_indices)


Selected Indices after NMS: [0 0 0]


In [6]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

# CUDA kernel for Non-Maximum Suppression
nms_kernel = """
__global__ void nms(float *boxes, int *selected_indices, int num_boxes, float threshold)
{
    int idx = threadIdx.x + blockDim.x * blockIdx.x;

    if (idx < num_boxes)
    {
        float x1 = boxes[idx * 4];
        float y1 = boxes[idx * 4 + 1];
        float x2 = boxes[idx * 4 + 2];
        float y2 = boxes[idx * 4 + 3];
        float area = (x2 - x1 + 1) * (y2 - y1 + 1);

        // Initialize selected index for the current box
        selected_indices[idx] = 1;

        for (int i = 0; i < num_boxes; ++i)
        {
            if (i != idx)
            {
                float xx1 = fmaxf(x1, boxes[i * 4]);
                float yy1 = fmaxf(y1, boxes[i * 4 + 1]);
                float xx2 = fminf(x2, boxes[i * 4 + 2]);
                float yy2 = fminf(y2, boxes[i * 4 + 3]);

                float w = fmaxf(0.0f, xx2 - xx1 + 1);
                float h = fmaxf(0.0f, yy2 - yy1 + 1);
                float overlap = (w * h) / area;

                if (overlap > threshold)
                {
                    // Suppress this box
                    selected_indices[idx] = 0;
                    return;
                }
            }
        }
    }
}
"""

def nms_cuda(boxes, threshold):
    num_boxes = boxes.shape[0]
    selected_indices = np.zeros(num_boxes, dtype=np.int32)

    # Allocate device memory
    boxes_gpu = cuda.mem_alloc(boxes.nbytes)
    selected_indices_gpu = cuda.mem_alloc(selected_indices.nbytes)

    # Copy data to device
    cuda.memcpy_htod(boxes_gpu, boxes)
    cuda.memcpy_htod(selected_indices_gpu, selected_indices)

    # Define block and grid dimensions
    block_size = 256
    grid_size = (num_boxes + block_size - 1) // block_size

    # Compile CUDA kernel
    mod = SourceModule(nms_kernel)
    nms_func = mod.get_function("nms")

    # Execute the kernel
    nms_func(boxes_gpu, selected_indices_gpu, np.int32(num_boxes), np.float32(threshold),
             block=(block_size, 1, 1), grid=(grid_size, 1))

    # Copy the result back to host
    cuda.memcpy_dtoh(selected_indices, selected_indices_gpu)

    return selected_indices

# Example usage:
boxes = np.array([[10, 10, 50, 50], [20, 20, 60, 60], [30, 30, 70, 70]], dtype=np.float32)
threshold = 0.5

selected_indices = nms_cuda(boxes, threshold)
print("Selected Indices after NMS:", selected_indices)


Selected Indices after NMS: [0 0 0]


In [7]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

nms_cuda_kernel = """
__global__ void nms_cuda(float *P, int *order, int *keep, int num_boxes, float thresh_iou, int box_stride)
{
    int idx = threadIdx.x + blockDim.x * blockIdx.x;

    if (idx < num_boxes)
    {
        int box_offset = idx * box_stride;

        float x1 = P[box_offset];
        float y1 = P[box_offset + 1];
        float x2 = P[box_offset + 2];
        float y2 = P[box_offset + 3];
        float area = (x2 - x1) * (y2 - y1);
        float score = P[box_offset + 4];

        int keep_index = atomicAdd(&keep[0], 1);
        keep_index *= box_stride;
        keep[keep_index] = idx;

        for (int i = 0; i < num_boxes; ++i)
        {
            if (i != idx)
            {
                int other_box_offset = order[i] * box_stride;
                float xx1 = fmaxf(x1, P[other_box_offset]);
                float yy1 = fmaxf(y1, P[other_box_offset + 1]);
                float xx2 = fminf(x2, P[other_box_offset + 2]);
                float yy2 = fminf(y2, P[other_box_offset + 3]);
                float w = fmaxf(0.0f, xx2 - xx1 + 1);
                float h = fmaxf(0.0f, yy2 - yy1 + 1);
                float inter = w * h;
                float other_area = (P[other_box_offset + 2] - P[other_box_offset]) * (P[other_box_offset + 3] - P[other_box_offset + 1]);
                float union_area = other_area - inter + area;
                float iou = inter / union_area;

                if (iou > thresh_iou)
                {
                    keep_index = atomicSub(&keep[0], 1) - 1;
                    keep_index *= box_stride;
                    keep[keep_index] = -1; // Mark box for removal
                    return;
                }
            }
        }
    }
}
"""

def nms_pytorch_pycuda(P, thresh_iou):
    num_boxes = P.shape[0]
    box_stride = P.shape[1]
    order = np.argsort(P[:, 4])[::-1]
    keep = np.zeros(num_boxes, dtype=np.int32)
    keep[0] = order[0]  # Keep the box with the highest score
    keep_index = np.zeros(1, dtype=np.int32)
    keep_gpu = cuda.mem_alloc(keep.nbytes)
    cuda.memcpy_htod(keep_gpu, keep)
    order_gpu = cuda.mem_alloc(order.nbytes)
    cuda.memcpy_htod(order_gpu, order)

    block_size = 256
    grid_size = (num_boxes + block_size - 1) // block_size

    mod = SourceModule(nms_cuda_kernel)
    nms_cuda_func = mod.get_function("nms_cuda")
    nms_cuda_func(cuda.InOut(P), order_gpu, keep_gpu, np.int32(num_boxes), np.float32(thresh_iou), np.int32(box_stride),
                  block=(block_size, 1, 1), grid=(grid_size, 1))

    cuda.memcpy_dtoh(keep, keep_gpu)
    filtered_boxes = [P[keep[i]*box_stride:(keep[i]+1)*box_stride] for i in range(len(keep)) if keep[i] != -1]
    return filtered_boxes

# Example usage:
import torch

P = torch.tensor([[10, 10, 50, 50, 0.8], [20, 20, 60, 60, 0.9], [30, 30, 70, 70, 0.85]], dtype=torch.float32)
thresh_iou = 0.5

filtered_boxes = nms_pytorch_pycuda(P.cpu().numpy(), thresh_iou)
print("Filtered Boxes after NMS:")
for box in filtered_boxes:
    print(box)

ValueError: ndarray is not contiguous

In [8]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

nms_cuda_kernel = """
__global__ void nms_cuda(float *P, int *order, int *keep, int num_boxes, float thresh_iou, int box_stride)
{
    int idx = threadIdx.x + blockDim.x * blockIdx.x;

    if (idx < num_boxes)
    {
        int box_offset = idx * box_stride;

        float x1 = P[box_offset];
        float y1 = P[box_offset + 1];
        float x2 = P[box_offset + 2];
        float y2 = P[box_offset + 3];
        float area = (x2 - x1) * (y2 - y1);
        float score = P[box_offset + 4];

        int keep_index = atomicAdd(&keep[0], 1);
        keep_index *= box_stride;
        keep[keep_index] = idx;

        for (int i = 0; i < num_boxes; ++i)
        {
            if (i != idx)
            {
                int other_box_offset = order[i] * box_stride;
                float xx1 = fmaxf(x1, P[other_box_offset]);
                float yy1 = fmaxf(y1, P[other_box_offset + 1]);
                float xx2 = fminf(x2, P[other_box_offset + 2]);
                float yy2 = fminf(y2, P[other_box_offset + 3]);
                float w = fmaxf(0.0f, xx2 - xx1 + 1);
                float h = fmaxf(0.0f, yy2 - yy1 + 1);
                float inter = w * h;
                float other_area = (P[other_box_offset + 2] - P[other_box_offset]) * (P[other_box_offset + 3] - P[other_box_offset + 1]);
                float union_area = other_area - inter + area;
                float iou = inter / union_area;

                if (iou > thresh_iou)
                {
                    keep_index = atomicSub(&keep[0], 1) - 1;
                    keep_index *= box_stride;
                    keep[keep_index] = -1; // Mark box for removal
                    return;
                }
            }
        }
    }
}
"""

def nms_pytorch_pycuda(P, thresh_iou):
    num_boxes = P.shape[0]
    box_stride = P.shape[1]
    order = np.argsort(P[:, 4])[::-1]
    keep = np.zeros(num_boxes, dtype=np.int32)
    keep[0] = order[0]  # Keep the box with the highest score
    keep_index = np.zeros(1, dtype=np.int32)
    keep_gpu = cuda.mem_alloc(keep.nbytes)
    cuda.memcpy_htod(keep_gpu, keep)
    order_contiguous = np.ascontiguousarray(order)  # Ensure order array is contiguous
    order_gpu = cuda.mem_alloc(order_contiguous.nbytes)
    cuda.memcpy_htod(order_gpu, order_contiguous)

    block_size = 256
    grid_size = (num_boxes + block_size - 1) // block_size

    mod = SourceModule(nms_cuda_kernel)
    nms_cuda_func = mod.get_function("nms_cuda")
    nms_cuda_func(cuda.InOut(P), order_gpu, keep_gpu, np.int32(num_boxes), np.float32(thresh_iou), np.int32(box_stride),
                  block=(block_size, 1, 1), grid=(grid_size, 1))

    cuda.memcpy_dtoh(keep, keep_gpu)
    filtered_boxes = [P[keep[i]*box_stride:(keep[i]+1)*box_stride] for i in range(len(keep)) if keep[i] != -1]
    return filtered_boxes

# Example usage:
import torch

P = torch.tensor([[10, 10, 50, 50, 0.8], [20, 20, 60, 60, 0.9], [30, 30, 70, 70, 0.85]], dtype=torch.float32)
thresh_iou = 0.5

filtered_boxes = nms_pytorch_pycuda(P.cpu().numpy(), thresh_iou)
print("Filtered Boxes after NMS:")
for box in filtered_boxes:
    print(box)


Filtered Boxes after NMS:
[]
[[10.   10.   50.   50.    0.8 ]
 [20.   20.   60.   60.    0.9 ]
 [30.   30.   70.   70.    0.85]]
[[10.   10.   50.   50.    0.8 ]
 [20.   20.   60.   60.    0.9 ]
 [30.   30.   70.   70.    0.85]]


/tmp/ipykernel_3318/3655634100.py:71: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(16): warning #177-D: variable "score" was declared but never referenced


  mod = SourceModule(nms_cuda_kernel)


In [9]:
import numpy as np
import cv2 as cv
 
img = cv.imread('../input_dir/camera01.tiff')
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
 
sift = cv.SIFT_create()
kp = sift.detect(gray,None)
 
img=cv.drawKeypoints(gray,kp,img)
 
cv.imwrite('sift_keypoints.jpg',img)

True

In [13]:
import cv2

img = cv2.imread('../input_dir/camera01.tiff')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

surf = cv2.xfeatures2d.SURF_create(8000)
keypoints, descriptors = surf.detectAndCompute(gray, None)

cv2.drawKeypoints(img, keypoints, img, (51, 163, 236),
                  cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('surf_keypoints', img)
cv2.waitKey()

AttributeError: module 'cv2' has no attribute 'xfeatures2d'